In [99]:
import pandas as pd
import numpy as np

train_data = pd.read_csv("train.csv")
#print(train_data["Name"][:10])
test_data = pd.read_csv("test.csv")
p_ID = test_data["PassengerId"]
train_data['Set']='train'
test_data['Set']='test'
combined = pd.concat([train_data, test_data], ignore_index=True)
#display(combined)

In [100]:
combined['Title'] = combined['Name'].str.extract('([A-Za-z]+)\.', expand=True)
#combined['Title'].unique()

title_reduction = {'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 
                   'Master': 'Master', 'Don': 'Mr', 'Rev': 'Mr',
                   'Dr': 'Mr', 'Mme': 'Miss', 'Ms': 'Miss',
                   'Major': 'Mr', 'Lady': 'Mrs', 'Sir': 'Mr',
                   'Mlle': 'Miss', 'Col': 'Mr', 'Capt': 'Mr',
                   'Countess': 'Mrs','Jonkheer': 'Mr',
                   'Dona': 'Mrs'}
combined['Title'] = combined['Title'].map(title_reduction)
#combined['Title'].unique()
pclass = combined.loc[combined.Fare.isnull(), 'Pclass'].values[0]
combined.loc[combined.Fare.isnull(), 'Fare'] = combined.loc[combined.Pclass==pclass, 'Fare'].mean()
#combined.loc[1043, :]

In [101]:
avg_girl_age = round(combined[(combined.Title=='Miss')&(combined.Parch!=0)]['Age'].mean())
avg_lady_age = round(combined[(combined.Title=="Miss") & (combined.Parch==0)]['Age'].mean())
avg_boy_age = round(combined[combined.Title=='Master']['Age'].mean())
avg_men_age = round(combined[combined.Title=='Mr']['Age'].mean())
avg_women_age = round(combined[combined.Title=='Mrs']['Age'].mean())
print(avg_girl_age, avg_lady_age, avg_boy_age, avg_men_age, avg_women_age)

12.0 27.0 5.0 33.0 37.0


In [102]:
#for title, age in combined.groupby('Title')['Age'].mean().iteritems():
#    print(title, age)
#    combined.loc[(combined.Title==title)&(combined.Age.isnull()), 'Age']=age
combined.loc[(combined.Title=='Miss')&(combined.Parch!=0)&(combined.Age.isnull()), 'Age']=avg_girl_age
combined.loc[(combined.Title=="Miss") & (combined.Parch==0)&(combined.Age.isnull()), 'Age']=avg_lady_age
combined.loc[(combined.Title=='Master')&(combined.Age.isnull()), 'Age']=avg_boy_age
combined.loc[(combined.Title=='Mr')&(combined.Age.isnull()), 'Age']=avg_men_age
combined.loc[(combined.Title=='Mrs')&(combined.Age.isnull()), 'Age']=avg_women_age
combined[combined.Age.isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Set,Title


In [103]:
combined = combined.drop(['Name', 'Ticket', 'Cabin', 'Embarked', "PassengerId"], axis=1)
#combined["Age"] = combined["Age"].apply(lambda x:np.log(x+1))
#combined["Fare"] = combined["Fare"].apply(lambda x:np.log(x+1))
#from sklearn.preprocessing import MinMaxScaler
#model = MinMaxScaler()
#print(combined["Age"].shape)
#combined["Age"] = model.fit_transform([combined["Age"]])[0, :]
#print(encoded.shape)
#combined["Fare"] = model.fit_transform([combined["Fare"]])[0, :]

In [83]:
encoded[0, :]

array([0., 0., 0., ..., 0., 0., 0.])

In [105]:
train = combined.loc[combined.Set=='train'].drop('Set', axis=1)
test = combined.loc[combined.Set=='test'].drop(['Set', 'Survived'], axis=1)
y_train = train['Survived']
train = train.drop('Survived', axis=1)

from sklearn.tree import DecisionTreeClassifier
#from sklearn.linear_model import LogisticRegression
clf = DecisionTreeClassifier(max_depth=20, random_state=0)
#clf = LogisticRegression(max_iter=1000, random_state=0)
clf.fit(pd.get_dummies(train), y_train)
predictions = clf.predict(pd.get_dummies(test))

print("Accuracy on training set: {}".format(clf.score(pd.get_dummies(train), y_train)))
result = pd.DataFrame({'PassengerId':p_ID, 'Survived':predictions}, dtype=int)
result.to_csv("result1.csv", index=False)

Accuracy on training set: 0.9820426487093153
